<a href="https://colab.research.google.com/github/Englishson0909/VerbTense01/blob/main/app1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [5]:
!pip install gTTS


In [7]:
import streamlit as st
import pandas as pd
import random
import io
from gtts import gTTS

# Load the irregular verbs data
data_url = "https://github.com/Hansukson/Application2/raw/main/irregular_verbs%20(1).csv"
verbs_df = pd.read_csv(data_url)

# Extract verbs data into a dictionary
verbs_data = verbs_df.set_index("present")[['past', 'p.p']].to_dict(orient="index")

# Correct Feedback List
correct_feedback = [
    "Correct! Fantastic job, {name}!",
    "Correct! Excellent work, {name}!",
    "Correct! You're doing great, {name}!",
    "Correct! Keep it up, {name} – you're unstoppable!",
    "Correct! Outstanding effort, {name}!",
    "Correct! Brilliant work, {name} – you nailed it!",
    "Correct! Amazing job, {name}! Keep shining!",
    "Correct! You're crushing it, {name}! Well done!",
    "Correct! Superb, {name}! You’re on fire!",
    "Correct! Perfect answer, {name}! Keep going strong!"
]

# Wrong Feedback List
wrong_feedback = [
    "Wrong! Nice try, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Almost there, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Don’t give up, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Good effort, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Mistakes happen, {name}. The correct forms are: {correct_past}, {correct_pp}.",
]

# Final Encouragement
final_encouragement = [
    "You’ve done so well today, {name}. I can truly see your effort. Keep learning and growing!",
    "I’m really proud of you, {name}. You’re moving forward step by step, and that’s what matters.",
    "You’ve made real progress, {name}! Remember, every new word is a step towards confidence.",
    "That was great work, {name}! Don’t forget how far you’ve come. Keep believing in yourself!",
    "Look at how much you’ve learned, {name}! I’m cheering you on every step of the way.",
    "Your dedication shows, {name}. Keep this up, and you’ll be amazed at your own growth.",
    "It’s wonderful to see you improve, {name}. Keep your spirits high and keep going!",
    "You’re getting stronger with every try, {name}. Your hard work is truly inspiring!",
    "Your commitment is shining through, {name}. Just keep at it, and you’ll do even better!",
    "I know it’s not always easy, {name}, but your progress is real. Stay motivated and keep pushing forward!"
]

def tts_play(verb_forms):
    tts_text = f"{verb_forms[0]}, {verb_forms[1]}, {verb_forms[2]}"
    tts = gTTS(tts_text)
    bytes_io = io.BytesIO()
    tts.write_to_fp(bytes_io)
    bytes_io.seek(0)
    return bytes_io

# Session state 초기화
if 'name' not in st.session_state:
    st.session_state.name = ""
if 'current_verb' not in st.session_state:
    st.session_state.current_verb = ""
if 'correct_count' not in st.session_state:
    st.session_state.correct_count = 0
if 'attempt_count' not in st.session_state:
    st.session_state.attempt_count = 0
if 'game_started' not in st.session_state:
    st.session_state.game_started = False
if 'final_stage' not in st.session_state:
    st.session_state.final_stage = False

st.title("VerbMaster: Learn Irregular Verbs! 🎯")
st.write("This app will help you learn irregular verbs.")

# 게임 시작 전 이름 입력
if not st.session_state.game_started and not st.session_state.final_stage:
    st.session_state.name = st.text_input("Your Name", value=st.session_state.name)
    if st.button("START"):
        if st.session_state.name.strip() == "":
            st.warning("Please enter your name to proceed!")
        else:
            st.session_state.game_started = True
            st.success(f"Welcome, {st.session_state.name}! Click 'SHOW ME A VERB' to begin.")

# 동사 보여주기
if st.session_state.game_started and not st.session_state.final_stage:
    if st.button("SHOW ME A VERB"):
        st.session_state.current_verb = random.choice(list(verbs_data.keys()))

    if st.session_state.current_verb:
        st.write("Present Verb:", st.session_state.current_verb)
        user_past = st.text_input("Enter Past Form", key='user_past')
        user_pp = st.text_input("Enter Past Participle", key='user_pp')

        if st.button("SUBMIT"):
            st.session_state.attempt_count += 1
            correct_past_str = verbs_data[st.session_state.current_verb]['past'].strip().lower()
            correct_pp_str = verbs_data[st.session_state.current_verb]['p.p'].strip().lower()
            correct_pp_list = [pp.strip() for pp in correct_pp_str.split('/')]

            upast = user_past.strip().lower()
            upp = user_pp.strip().lower()

            audio_bytes = tts_play([st.session_state.current_verb, correct_past_str, correct_pp_str])

            if upast == correct_past_str and upp in correct_pp_list:
                st.session_state.correct_count += 1
                feedback = random.choice(correct_feedback).format(name=st.session_state.name)
            else:
                feedback = random.choice(wrong_feedback).format(
                    name=st.session_state.name,
                    correct_past=correct_past_str,
                    correct_pp=correct_pp_str
                )

            score = f"Your Score: {st.session_state.correct_count} / {st.session_state.attempt_count}"

            st.write("Feedback:", feedback)
            st.write("Recheck:", f"{st.session_state.current_verb} {correct_past_str} {correct_pp_str}")
            st.write(score)
            st.audio(audio_bytes, format='audio/mp3')

            # 계속하기 및 끝내기
            col1, col2 = st.columns(2)
            with col1:
                if st.button("IF YOU WANT TO CONTINUE, CLICK HERE!"):
                    # 다음 동사 준비
                    st.session_state.user_past = ""
                    st.session_state.user_pp = ""
                    st.session_state.current_verb = ""
                    st.experimental_rerun()

            with col2:
                if st.button("IF YOU WANT TO END THIS APP, CLICK HERE!"):
                    st.session_state.final_stage = True
                    st.experimental_rerun()

# 게임 종료 후 메시지
if st.session_state.final_stage:
    st.markdown("### THE END")
    st.markdown(random.choice(final_encouragement).format(name=st.session_state.name))

2024-12-26 14:23:18.635 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-26 14:23:18.636 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-26 14:23:18.639 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-26 14:23:18.641 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-26 14:23:18.644 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-26 14:23:18.645 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-26 14:23:18.646 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-26 14:23:18.648 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [9]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [10]:
import gradio as gr
import pandas as pd
import random
import io
from gtts import gTTS

# ----- 데이터 불러오기 및 전처리 -----
data_url = "https://github.com/Hansukson/Application2/raw/main/irregular_verbs%20(1).csv"
verbs_df = pd.read_csv(data_url)
# present 컬럼을 인덱스로, past와 p.p만 뽑아서 dictionary 형태로 변환
verbs_data = verbs_df.set_index("present")[['past', 'p.p']].to_dict(orient="index")

# ----- 피드백 문구들 -----
correct_feedback = [
    "Correct! Fantastic job, {name}!",
    "Correct! Excellent work, {name}!",
    "Correct! You're doing great, {name}!",
    "Correct! Keep it up, {name} – you're unstoppable!",
    "Correct! Outstanding effort, {name}!",
    "Correct! Brilliant work, {name} – you nailed it!",
    "Correct! Amazing job, {name}! Keep shining!",
    "Correct! You're crushing it, {name}! Well done!",
    "Correct! Superb, {name}! You’re on fire!",
    "Correct! Perfect answer, {name}! Keep going strong!"
]

wrong_feedback = [
    "Wrong! Nice try, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Almost there, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Don’t give up, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Good effort, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Mistakes happen, {name}. The correct forms are: {correct_past}, {correct_pp}.",
]

final_encouragement = [
    "You’ve done so well today, {name}. I can truly see your effort. Keep learning and growing!",
    "I’m really proud of you, {name}. You’re moving forward step by step, and that’s what matters.",
    "You’ve made real progress, {name}! Remember, every new word is a step towards confidence.",
    "That was great work, {name}! Don’t forget how far you’ve come. Keep believing in yourself!",
    "Look at how much you’ve learned, {name}! I’m cheering you on every step of the way.",
    "Your dedication shows, {name}. Keep this up, and you’ll be amazed at your own growth.",
    "It’s wonderful to see you improve, {name}. Keep your spirits high and keep going!",
    "You’re getting stronger with every try, {name}. Your hard work is truly inspiring!",
    "Your commitment is shining through, {name}. Just keep at it, and you’ll do even better!",
    "I know it’s not always easy, {name}, but your progress is real. Stay motivated and keep pushing forward!"
]

# ----- 주요 함수 정의 -----
def generate_verb(name, current_verb, correct_count, attempt_count, final_stage):
    """
    'SHOW ME A VERB' 버튼 클릭 시 실행:
    - 새로운 동사를 랜덤으로 뽑아서 반환
    - 이미 final_stage면 새 동사를 안 뽑고 메시지 반환
    """
    if final_stage:
        return (
            "Game is already finished. Please press 'END GAME' or refresh.",
            current_verb,  # 그대로 유지
            correct_count,
            attempt_count,
            final_stage
        )
    new_verb = random.choice(list(verbs_data.keys()))
    return (
        f"New Verb: {new_verb}",
        new_verb,
        correct_count,
        attempt_count,
        final_stage
    )

def check_answer(name, user_past, user_pp, current_verb, correct_count, attempt_count, final_stage):
    """
    'SUBMIT' 버튼 클릭 시 실행:
    - 정답 판별 후 피드백, 오디오, 스코어 등을 반환
    """
    # 아직 시작 전이거나 Verb가 없는 상태라면
    if final_stage:
        return (
            "Game is already finished. Please press 'END GAME' or refresh.",
            current_verb,
            correct_count,
            attempt_count,
            final_stage,
            None
        )
    if not current_verb:
        return (
            "No verb to check. Please click 'SHOW ME A VERB' first!",
            current_verb,
            correct_count,
            attempt_count,
            final_stage,
            None
        )

    correct_past_str = verbs_data[current_verb]['past'].strip().lower()
    correct_pp_str = verbs_data[current_verb]['p.p'].strip().lower()
    correct_pp_list = [pp.strip() for pp in correct_pp_str.split('/')]

    upast = user_past.strip().lower()
    upp = user_pp.strip().lower()

    # 시도 횟수 증가
    attempt_count += 1

    # TTS (정답 읽어주기)
    tts_text = f"{current_verb}, {correct_past_str}, {correct_pp_str}"
    tts = gTTS(tts_text)
    bytes_io = io.BytesIO()
    tts.write_to_fp(bytes_io)
    bytes_io.seek(0)

    # 정답 여부 확인
    if upast == correct_past_str and upp in correct_pp_list:
        correct_count += 1
        feedback = random.choice(correct_feedback).format(name=name)
    else:
        feedback = random.choice(wrong_feedback).format(
            name=name,
            correct_past=correct_past_str,
            correct_pp=correct_pp_str
        )

    score = f"Your Score: {correct_count} / {attempt_count}"
    recheck = f"Recheck: {current_verb} {correct_past_str} {correct_pp_str}"
    combined_msg = f"{feedback}\n{recheck}\n{score}"

    return (
        combined_msg,                      # 피드백 메시지
        current_verb,                     # verb는 그대로 유지
        correct_count,
        attempt_count,
        final_stage,
        (bytes_io.getvalue(), "audio/mp3")  # 오디오
    )

def end_game(name, current_verb, correct_count, attempt_count, final_stage):
    """
    'END GAME' 버튼 클릭 시 실행:
    - 게임 종료 메시지와 함께 final_stage를 True로 설정
    """
    final_stage = True
    message = "### THE END\n\n" + random.choice(final_encouragement).format(name=name)
    return (
        message,
        current_verb,
        correct_count,
        attempt_count,
        final_stage
    )

# ----- Gradio UI 구성 -----
def gradio_app():
    with gr.Blocks() as demo:
        # state(상태) 초기값
        name = gr.Textbox(label="Your Name", value="", interactive=True)
        current_verb = gr.State("")
        correct_count = gr.State(0)
        attempt_count = gr.State(0)
        final_stage = gr.State(False)

        # 1) 새 동사 뽑기
        generate_button = gr.Button("SHOW ME A VERB")
        generate_output = gr.Textbox(label="Generated Verb", interactive=False)

        # 2) 사용자 입력 (과거형, 과거분사)
        user_past = gr.Textbox(label="Enter Past Form")
        user_pp = gr.Textbox(label="Enter Past Participle")

        # 3) 제출 버튼 및 피드백 출력
        submit_button = gr.Button("SUBMIT")
        feedback_output = gr.Textbox(label="Feedback", interactive=False, lines=5)
        audio_output = gr.Audio(label="TTS (정답 듣기)")

        # 4) 종료 버튼 및 종료 메시지
        end_button = gr.Button("END GAME")
        end_output = gr.Textbox(label="Final Message", interactive=False, lines=5)

        # 버튼 클릭 시 동작 연결
        generate_button.click(
            fn=generate_verb,
            inputs=[name, current_verb, correct_count, attempt_count, final_stage],
            outputs=[generate_output, current_verb, correct_count, attempt_count, final_stage]
        )

        submit_button.click(
            fn=check_answer,
            inputs=[name, user_past, user_pp, current_verb, correct_count, attempt_count, final_stage],
            outputs=[feedback_output, current_verb, correct_count, attempt_count, final_stage, audio_output]
        )

        end_button.click(
            fn=end_game,
            inputs=[name, current_verb, correct_count, attempt_count, final_stage],
            outputs=[end_output, current_verb, correct_count, attempt_count, final_stage]
        )

    return demo

# 직접 실행 시 서버 실행
if __name__ == "__main__":
    app = gradio_app()
    app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3171d261381f20dba2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
import gradio as gr
import pandas as pd
import random
import io
from gtts import gTTS

# ----- 데이터 불러오기 및 전처리 -----
# URL이 동작하지 않거나, 네트워크 문제로 CSV 불러오기 실패 시를 대비해 try-except 처리
data_url = "https://github.com/Hansukson/Application2/raw/main/irregular_verbs%20(1).csv"

try:
    verbs_df = pd.read_csv(data_url)
except Exception as e:
    # 데이터를 불러오지 못하면 예시로 아래 dictionary를 사용 (직접 몇 가지 예시만 포함)
    # 실제로는 CSV가 정상적으로 로드되면 이 부분은 무시됩니다.
    print("Failed to load CSV from GitHub. Using fallback data. Error:", e)
    data = {
        "present": ["be", "begin", "come"],
        "past": ["was/were", "began", "came"],
        "p.p": ["been", "begun", "come"]
    }
    verbs_df = pd.DataFrame(data)

# present 컬럼을 인덱스로, past와 p.p만 뽑아서 dictionary 형태로 변환
verbs_data = verbs_df.set_index("present")[['past', 'p.p']].to_dict(orient="index")

# ----- 피드백 문구들 -----
correct_feedback = [
    "Correct! Fantastic job, {name}!",
    "Correct! Excellent work, {name}!",
    "Correct! You're doing great, {name}!",
    "Correct! Keep it up, {name} – you're unstoppable!",
    "Correct! Outstanding effort, {name}!",
    "Correct! Brilliant work, {name} – you nailed it!",
    "Correct! Amazing job, {name}! Keep shining!",
    "Correct! You're crushing it, {name}! Well done!",
    "Correct! Superb, {name}! You’re on fire!",
    "Correct! Perfect answer, {name}! Keep going strong!"
]

wrong_feedback = [
    "Wrong! Nice try, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Almost there, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Don’t give up, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Good effort, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Mistakes happen, {name}. The correct forms are: {correct_past}, {correct_pp}.",
]

final_encouragement = [
    "You’ve done so well today, {name}. I can truly see your effort. Keep learning and growing!",
    "I’m really proud of you, {name}. You’re moving forward step by step, and that’s what matters.",
    "You’ve made real progress, {name}! Remember, every new word is a step towards confidence.",
    "That was great work, {name}! Don’t forget how far you’ve come. Keep believing in yourself!",
    "Look at how much you’ve learned, {name}! I’m cheering you on every step of the way.",
    "Your dedication shows, {name}. Keep this up, and you’ll be amazed at your own growth.",
    "It’s wonderful to see you improve, {name}. Keep your spirits high and keep going!",
    "You’re getting stronger with every try, {name}. Your hard work is truly inspiring!",
    "Your commitment is shining through, {name}. Just keep at it, and you’ll do even better!",
    "I know it’s not always easy, {name}, but your progress is real. Stay motivated and keep pushing forward!"
]

# ----- 주요 함수 정의 -----
def generate_verb(
    name,
    current_verb,
    correct_count,
    attempt_count,
    final_stage
):
    """
    'SHOW ME A VERB' 버튼 클릭 시 실행:
    - 새로운 동사를 랜덤으로 뽑아서 반환
    - 이미 final_stage면 새 동사를 안 뽑고 메시지 반환
    """
    # 게임이 끝난 상태라면
    if final_stage:
        return (
            "Game is already finished. Please press 'END GAME' or refresh.",
            current_verb,
            correct_count,
            attempt_count,
            final_stage
        )

    # 동사 목록이 비어있을 경우 처리
    if not verbs_data:
        return (
            "No verbs available! (Check CSV loading)",
            current_verb,
            correct_count,
            attempt_count,
            final_stage
        )

    new_verb = random.choice(list(verbs_data.keys()))
    return (
        f"New Verb: {new_verb}",
        new_verb,
        correct_count,
        attempt_count,
        final_stage
    )


def check_answer(
    name,
    user_past,
    user_pp,
    current_verb,
    correct_count,
    attempt_count,
    final_stage
):
    """
    'SUBMIT' 버튼 클릭 시 실행:
    - 정답 판별 후 피드백, 오디오, 스코어 등을 반환
    """
    # 이미 게임이 끝난 상태라면
    if final_stage:
        return (
            "Game is already finished. Please press 'END GAME' or refresh.",
            current_verb,
            correct_count,
            attempt_count,
            final_stage,
            None
        )

    # 동사가 아직 선정되지 않았다면
    if not current_verb:
        return (
            "No verb to check. Please click 'SHOW ME A VERB' first!",
            current_verb,
            correct_count,
            attempt_count,
            final_stage,
            None
        )

    # CSV 내 동사를 찾을 수 없을 경우(드물지만)
    if current_verb not in verbs_data:
        return (
            f"Error: '{current_verb}' not found in data. Please click 'SHOW ME A VERB' again.",
            "",
            correct_count,
            attempt_count,
            final_stage,
            None
        )

    correct_past_str = verbs_data[current_verb]['past'].strip().lower()
    correct_pp_str = verbs_data[current_verb]['p.p'].strip().lower()
    correct_pp_list = [pp.strip() for pp in correct_pp_str.split('/')]

    upast = user_past.strip().lower()
    upp = user_pp.strip().lower()

    # 시도 횟수 증가
    attempt_count += 1

    # TTS(정답 읽어주기)용 텍스트
    tts_text = f"{current_verb}, {correct_past_str}, {correct_pp_str}"

    # gTTS 오류 방지를 위해, 혹시나 빈 문자열이면 기본 안내 메시지 세팅
    if not tts_text.strip():
        tts_text = "No text for TTS."

    tts = gTTS(tts_text)
    bytes_io = io.BytesIO()
    tts.write_to_fp(bytes_io)
    bytes_io.seek(0)

    # 정답 여부 확인
    if upast == correct_past_str and upp in correct_pp_list:
        correct_count += 1
        feedback = random.choice(correct_feedback).format(name=name or "User")
    else:
        feedback = random.choice(wrong_feedback).format(
            name=name or "User",
            correct_past=correct_past_str,
            correct_pp=correct_pp_str
        )

    score = f"Your Score: {correct_count} / {attempt_count}"
    recheck = f"Recheck: {current_verb} -> {correct_past_str}, {correct_pp_str}"
    combined_msg = f"{feedback}\n{recheck}\n{score}"

    return (
        combined_msg,
        current_verb,
        correct_count,
        attempt_count,
        final_stage,
        (bytes_io.getvalue(), "audio/mp3")
    )


def end_game(
    name,
    current_verb,
    correct_count,
    attempt_count,
    final_stage
):
    """
    'END GAME' 버튼 클릭 시 실행:
    - 게임 종료 메시지와 함께 final_stage를 True로 설정
    """
    final_stage = True
    message = "### THE END\n\n"
    message += random.choice(final_encouragement).format(name=name or "User")
    return (
        message,
        current_verb,
        correct_count,
        attempt_count,
        final_stage
    )

# ----- Gradio UI 구성 -----
def gradio_app():
    with gr.Blocks() as demo:
        gr.Markdown("## Irregular Verb Learning App (Gradio Version)")

        # 상태(state) 초기값
        name = gr.Textbox(label="Your Name", value="")
        current_verb = gr.State("")
        correct_count = gr.State(0)
        attempt_count = gr.State(0)
        final_stage = gr.State(False)

        with gr.Row():
            generate_button = gr.Button("SHOW ME A VERB")
            end_button = gr.Button("END GAME")

        generate_output = gr.Textbox(label="Generated Verb", interactive=False)

        user_past = gr.Textbox(label="Enter Past Form")
        user_pp = gr.Textbox(label="Enter Past Participle")

        submit_button = gr.Button("SUBMIT")
        feedback_output = gr.Textbox(label="Feedback", interactive=False, lines=5)
        audio_output = gr.Audio(label="TTS (정답 듣기)", interactive=False)

        # 콜백 연결
        generate_button.click(
            fn=generate_verb,
            inputs=[name, current_verb, correct_count, attempt_count, final_stage],
            outputs=[generate_output, current_verb, correct_count, attempt_count, final_stage]
        )

        submit_button.click(
            fn=check_answer,
            inputs=[name, user_past, user_pp, current_verb, correct_count, attempt_count, final_stage],
            outputs=[feedback_output, current_verb, correct_count, attempt_count, final_stage, audio_output]
        )

        end_button.click(
            fn=end_game,
            inputs=[name, current_verb, correct_count, attempt_count, final_stage],
            outputs=[feedback_output, current_verb, correct_count, attempt_count, final_stage]
        )

    return demo

# 직접 실행 시 서버 실행
if __name__ == "__main__":
    app = gradio_app()
    app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04081f21659d85461e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
import gradio as gr
import random
import io
from gtts import gTTS

# ----- 간단한 예시 데이터 (CSV 대신 사용) -----
verbs_data = {
    "be":    {"past": "was/were", "p.p": "been"},
    "begin": {"past": "began",    "p.p": "begun"},
    "come":  {"past": "came",     "p.p": "come"}
}

# ----- 피드백 문구들 -----
correct_feedback = [
    "Correct! Fantastic job, {name}!",
    "Correct! Excellent work, {name}!",
    "Correct! You're doing great, {name}!",
    "Correct! Keep it up, {name} – you're unstoppable!",
    "Correct! Outstanding effort, {name}!",
    "Correct! Brilliant work, {name} – you nailed it!",
    "Correct! Amazing job, {name}! Keep shining!",
    "Correct! You're crushing it, {name}! Well done!",
    "Correct! Superb, {name}! You’re on fire!",
    "Correct! Perfect answer, {name}! Keep going strong!"
]

wrong_feedback = [
    "Wrong! Nice try, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Almost there, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Don’t give up, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Good effort, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Mistakes happen, {name}. The correct forms are: {correct_past}, {correct_pp}.",
]

final_encouragement = [
    "You’ve done so well today, {name}. I can truly see your effort. Keep learning and growing!",
    "I’m really proud of you, {name}. You’re moving forward step by step, and that’s what matters.",
    "You’ve made real progress, {name}! Remember, every new word is a step towards confidence.",
    "That was great work, {name}! Don’t forget how far you’ve come. Keep believing in yourself!",
    "Look at how much you’ve learned, {name}! I’m cheering you on every step of the way.",
    "Your dedication shows, {name}. Keep this up, and you’ll be amazed at your own growth.",
    "It’s wonderful to see you improve, {name}. Keep your spirits high and keep going!",
    "You’re getting stronger with every try, {name}. Your hard work is truly inspiring!",
    "Your commitment is shining through, {name}. Just keep at it, and you’ll do even better!",
    "I know it’s not always easy, {name}, but your progress is real. Stay motivated and keep pushing forward!"
]

# ----- 주요 함수들 -----
def generate_verb(name, current_verb, correct_count, attempt_count, final_stage):
    """
    'SHOW ME A VERB' 버튼 클릭 시:
    - 게임이 끝나지 않았다면, 임의의 동사를 뽑아 반환
    - 게임이 이미 끝났다면, 메세지 반환
    """
    if final_stage:
        return (
            "Game is already finished. Please press 'END GAME' or refresh.",
            current_verb,
            correct_count,
            attempt_count,
            final_stage
        )

    new_verb = random.choice(list(verbs_data.keys()))
    return (
        f"New Verb: {new_verb}",
        new_verb,
        correct_count,
        attempt_count,
        final_stage
    )

def check_answer(name, user_past, user_pp, current_verb, correct_count, attempt_count, final_stage):
    """
    'SUBMIT' 버튼 클릭 시:
    - 정답/오답 체크 후, 피드백과 오디오(TTS) 반환
    """
    if final_stage:
        return (
            "Game is already finished. Please press 'END GAME' or refresh.",
            current_verb,
            correct_count,
            attempt_count,
            final_stage,
            None
        )
    if not current_verb:
        return (
            "No verb to check. Please click 'SHOW ME A VERB' first!",
            current_verb,
            correct_count,
            attempt_count,
            final_stage,
            None
        )

    # 시도 횟수 증가
    attempt_count += 1

    correct_past = verbs_data[current_verb]['past'].lower()
    correct_pp = verbs_data[current_verb]['p.p'].lower()
    # 혹시 여러 형태가 있을 수 있어, / 로 구분된 것이나 여기서는 간단히 단일형만 사용
    correct_pp_list = [pp.strip() for pp in correct_pp.split('/')]

    upast = user_past.strip().lower()
    upp = user_pp.strip().lower()

    # TTS 오디오 생성
    tts_text = f"{current_verb}, {correct_past}, {correct_pp}"
    tts = gTTS(tts_text)
    bytes_io = io.BytesIO()
    tts.write_to_fp(bytes_io)
    bytes_io.seek(0)

    # 정답 여부
    if upast == correct_past and upp in correct_pp_list:
        correct_count += 1
        feedback = random.choice(correct_feedback).format(name=name or "User")
    else:
        feedback = random.choice(wrong_feedback).format(
            name=name or "User",
            correct_past=verbs_data[current_verb]['past'],
            correct_pp=verbs_data[current_verb]['p.p']
        )

    score = f"Your Score: {correct_count} / {attempt_count}"
    recheck = f"Recheck: {current_verb} -> {verbs_data[current_verb]['past']}, {verbs_data[current_verb]['p.p']}"
    combined_msg = f"{feedback}\n{recheck}\n{score}"

    return (
        combined_msg,
        current_verb,
        correct_count,
        attempt_count,
        final_stage,
        (bytes_io.getvalue(), "audio/mp3")  # (오디오 바이트, 포맷)
    )

def end_game(name, current_verb, correct_count, attempt_count, final_stage):
    """
    'END GAME' 버튼 클릭 시:
    - 종료 메시지와 함께 final_stage를 True로 세팅
    """
    final_stage = True
    message = "### THE END\n\n" + random.choice(final_encouragement).format(name=name or "User")
    return (
        message,
        current_verb,
        correct_count,
        attempt_count,
        final_stage
    )

# ----- Gradio 인터페이스 구성 -----
def gradio_app():
    with gr.Blocks() as demo:
        gr.Markdown("## Irregular Verb Learning (Minimal Example)")

        name = gr.Textbox(label="Your Name", value="")
        current_verb = gr.State("")
        correct_count = gr.State(0)
        attempt_count = gr.State(0)
        final_stage = gr.State(False)

        with gr.Row():
            generate_button = gr.Button("SHOW ME A VERB")
            end_button = gr.Button("END GAME")

        generate_output = gr.Textbox(label="Generated Verb", interactive=False)

        user_past = gr.Textbox(label="Enter Past Form")
        user_pp = gr.Textbox(label="Enter Past Participle")

        submit_button = gr.Button("SUBMIT")
        feedback_output = gr.Textbox(label="Feedback", interactive=False, lines=5)
        audio_output = gr.Audio(label="TTS (정답 듣기)", interactive=False)

        # 버튼 클릭 시 각 함수 연결
        generate_button.click(
            fn=generate_verb,
            inputs=[name, current_verb, correct_count, attempt_count, final_stage],
            outputs=[generate_output, current_verb, correct_count, attempt_count, final_stage]
        )

        submit_button.click(
            fn=check_answer,
            inputs=[name, user_past, user_pp, current_verb, correct_count, attempt_count, final_stage],
            outputs=[feedback_output, current_verb, correct_count, attempt_count, final_stage, audio_output]
        )

        end_button.click(
            fn=end_game,
            inputs=[name, current_verb, correct_count, attempt_count, final_stage],
            outputs=[feedback_output, current_verb, correct_count, attempt_count, final_stage]
        )

    return demo

if __name__ == "__main__":
    app = gradio_app()
    app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://44a2b7fbb03306ea8e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
!pip install gradio gtts pandas

import gradio as gr
import pandas as pd
import random
import os
import tempfile
from gtts import gTTS

# Load the irregular verbs data
data_url = "https://github.com/Hansukson/Application2/raw/main/irregular_verbs%20(1).csv"
verbs_df = pd.read_csv(data_url)

# Extract verbs data into a dictionary
verbs_data = verbs_df.set_index("present")[['past', 'p.p']].to_dict(orient="index")

# Correct Feedback List (unchanged)
correct_feedback = [
    "Correct! Fantastic job, {name}!",
    "Correct! Excellent work, {name}!",
    "Correct! You're doing great, {name}!",
    "Correct! Keep it up, {name} – you're unstoppable!",
    "Correct! Outstanding effort, {name}!",
    "Correct! Brilliant work, {name} – you nailed it!",
    "Correct! Amazing job, {name}! Keep shining!",
    "Correct! You're crushing it, {name}! Well done!",
    "Correct! Superb, {name}! You’re on fire!",
    "Correct! Perfect answer, {name}! Keep going strong!"
]

# Wrong Feedback List (unchanged)
wrong_feedback = [
    "Wrong! Nice try, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Almost there, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Don’t give up, {name}. The correct forms are: {correct_past}, {correct_pp}.",
    "Wrong! Good effort, {name}. The correct answer is: {correct_past}, {correct_pp}.",
    "Wrong! Mistakes happen, {name}. The correct forms are: {correct_past}, {correct_pp}.",
]

# Final Encouragement Feedback - 더 따뜻하고 인간적인 응원 추가
final_encouragement = [
    "You’ve done so well today, {name}. I can truly see your effort. Keep learning and growing!",
    "I’m really proud of you, {name}. You’re moving forward step by step, and that’s what matters.",
    "You’ve made real progress, {name}! Remember, every new word is a step towards confidence.",
    "That was great work, {name}! Don’t forget how far you’ve come. Keep believing in yourself!",
    "Look at how much you’ve learned, {name}! I’m cheering you on every step of the way.",
    "Your dedication shows, {name}. Keep this up, and you’ll be amazed at your own growth.",
    "It’s wonderful to see you improve, {name}. Keep your spirits high and keep going!",
    "You’re getting stronger with every try, {name}. Your hard work is truly inspiring!",
    "Your commitment is shining through, {name}. Just keep at it, and you’ll do even better!",
    "I know it’s not always easy, {name}, but your progress is real. Stay motivated and keep pushing forward!"
]

def tts_play(verb_forms):
    tts_text = f"{verb_forms[0]}, {verb_forms[1]}, {verb_forms[2]}"
    temp_audio_file = os.path.join(tempfile.gettempdir(), "verb_audio.mp3")
    tts = gTTS(tts_text)
    tts.save(temp_audio_file)
    return temp_audio_file

current_verb = ""
correct_count = 0
attempt_count = 0

def start_game(name):
    if name.strip() == "":
        return "Please enter your name to proceed!", gr.update(visible=False)
    return f"Welcome, {name}! Click 'SHOW ME A VERB' to begin.", gr.update(visible=True)

def show_random_verb():
    global current_verb
    current_verb = random.choice(list(verbs_data.keys()))
    return current_verb

def check_answer(name, user_past, user_past_participle):
    global correct_count, attempt_count, current_verb
    attempt_count += 1

    correct_past_str = verbs_data[current_verb]['past'].strip().lower()
    correct_pp_str = verbs_data[current_verb]['p.p'].strip().lower()
    correct_pp_list = [pp.strip() for pp in correct_pp_str.split('/')]

    user_past = user_past.strip().lower()
    user_past_participle = user_past_participle.strip().lower()

    audio_file = tts_play([current_verb, correct_past_str, correct_pp_str])

    if user_past == correct_past_str and user_past_participle in correct_pp_list:
        correct_count += 1
        feedback = random.choice(correct_feedback).format(name=name)
    else:
        feedback = random.choice(wrong_feedback).format(name=name, correct_past=correct_past_str, correct_pp=correct_pp_str)

    score = f"Your Score: {correct_count} / {attempt_count}"

    recheck_msg = f"{current_verb} {correct_past_str} {correct_pp_str}"

    return feedback, gr.update(value=recheck_msg, visible=True), score, audio_file, gr.update(visible=True), gr.update(visible=True)

def final_feedback(name):
    return f"### THE END\n{random.choice(final_encouragement).format(name=name)}"

def reset_inputs():
    return "", "", ""

def verb_game():
    with gr.Blocks() as app:
        gr.Markdown("# VerbMaster: Learn Irregular Verbs! 🎯")
        gr.Markdown("This app will help you learn irregular verbs.")

        name_input = gr.Textbox(label="Your Name")
        start_button = gr.Button("START")
        welcome_output = gr.Markdown()

        show_verb_button = gr.Button("SHOW ME A VERB", visible=False)

        present_verb_output = gr.Textbox(label="Present Verb", interactive=False)
        user_past_input = gr.Textbox(label="Enter Past Form")
        user_pp_input = gr.Textbox(label="Enter Past Participle")

        submit_button = gr.Button("SUBMIT", visible=False)
        feedback_output = gr.Textbox(label="Feedback", interactive=False)
        recheck_output = gr.Textbox(label="Recheck", interactive=False, visible=False)
        score_output = gr.Textbox(label="Score", interactive=False)

        audio_button = gr.Button("NOT SURE HOW TO SAY IT? HEAR IT HERE!", visible=False)
        tts_output = gr.Audio(label="Audio Feedback", visible=False)

        continue_button = gr.Button("IF YOU WANT TO CONTINUE, CLICK HERE!", visible=False)
        end_button = gr.Button("IF YOU WANT TO END THIS APP, CLICK HERE!", visible=False)

        final_feedback_output = gr.Markdown(visible=False)

        # START 버튼 클릭 -> SHOW ME A VERB 활성화
        start_button.click(start_game, inputs=name_input, outputs=[welcome_output, show_verb_button])

        # SHOW ME A VERB 클릭 -> 동사 표시
        show_verb_button.click(show_random_verb, outputs=present_verb_output)

        # Past Participle 입력 시 Submit 버튼 표시
        user_pp_input.change(
            lambda val: gr.update(visible=True) if val.strip() else gr.update(visible=False),
            inputs=user_pp_input,
            outputs=submit_button
        )

        # SUBMIT 클릭 -> feedback, recheck, score, 오디오, 발음 버튼, continue, end 버튼
        submit_button.click(
            check_answer,
            inputs=[name_input, user_past_input, user_pp_input],
            outputs=[feedback_output, recheck_output, score_output, tts_output, audio_button, continue_button]
        )
        submit_button.click(lambda: gr.update(visible=True), outputs=end_button)

        # 발음 버튼 클릭 -> 오디오 표시
        audio_button.click(lambda: gr.update(visible=True), outputs=tts_output)

        # Continue 버튼 -> 입력 리셋, 새로운 동사
        continue_button.click(reset_inputs, outputs=[user_past_input, user_pp_input, feedback_output])
        continue_button.click(show_random_verb, outputs=present_verb_output)

        # End 버튼 클릭 시 Final Feedback 표시
        end_button.click(final_feedback, inputs=name_input, outputs=final_feedback_output)
        end_button.click(lambda: gr.update(visible=True), outputs=final_feedback_output)

    return app

app = verb_game()
app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b55f81695d5e4df26e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
